In [2]:
!pip install datasets

  Using cached filelock-3.20.0-py3-none-any.whl.metadata (2.1 kB)
  Using cached numpy-2.3.4-cp313-cp313-win_amd64.whl.metadata (60 kB)
  Using cached pandas-2.3.3-cp313-cp313-win_amd64.whl.metadata (19 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached click-8.3.0-py3-none-any.whl.metadata (2.6 kB)
   ---------------------------------------- 0.0/2.9 MB ? eta -:--:--
   --------------------- ------------------ 1.6/2.9 MB 7.6 MB/s eta 0:00:01
   ---------------------------------------- 2.9/2.9 MB 6.9 MB/s  0:00:00
Using cached numpy-2.3.4-cp313-cp313-win_amd64.whl (12.8 MB)
   ---------------------------------------- 0.0/28.0 MB ? eta -:--:--
   -- ------------------------------------- 1.6/28.0 MB 8.1 MB/s eta 0:00:04
   ---- ----------------------------------- 3.4/28.0 MB 8.5 MB/s eta 0:00:03
   ------- -------------------------------- 5.2/28.0 MB 8.5 MB/s eta 0:00:03
   ---------- -----------


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


# Cases Similarity DS Task

## The Task
* Part #1 - Given 2 cases ids - find a similarity score [0-1] that measures how similar the cases are
* Part #2 - Build a multiclass classification model to predict domain for each case

## Data
* Dataframe with 3 columns containing past legal cases
  * id - unique id of the case
  * text - the actual text of the case
  * domain - a categorical value representing the domain which this case belongs to

In [3]:
!pip install pandas


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
import pandas as pd
import numpy as np

# Reading the Legal Cases Dataframe
* case_id - represents a unique id for each case
* text - represents the complaint text for each case
* domain - the domain name of which this case belongs to

In [5]:
from datasets import load_dataset
dataset = load_dataset('darrow-ai/legal-task')
dataset = dataset['train'].to_pandas()
dataset.head()

C:\Users\ronyg\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\ronyg\AppData\Local\Programs\Python\Python313\Lib\site-packages\huggingface_hub\file_download.py:121: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\ronyg\.cache\huggingface\hub\datasets--darrow-ai--legal-task. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an a

,id,text,domain
0,r-e4EYcBD5gMZwcz41zP,UNITED STATES DISTRICT COURT \nEASTERN DISTRIC...,consumer fraud
1,i9H5DocBD5gMZwcztj0y,IN THE UNITED STATES DISTRICT COURT \nFOR THE ...,privacy
2,SMn3DYcBD5gMZwcz-hwH,IN THE UNITED STATES DISTRICT COURT\n FOR THE ...,privacy
3,GMIWDYcBD5gMZwczDQBb,Case No. _______________ \n \n \nCLASS ACTION ...,criminal & enforcement
4,lELw_IgBF5pVm5zYONwC,UNITED STATES DISTRICT COURT \n SOUTHERN DISTR...,consumer fraud


In [8]:
dataset.id.count()

np.int64(1204)

In [7]:
dataset.groupby('domain').agg(
    total_count=('id', 'count'),
    distinct_count=('id', 'nunique')
)


,total_count,distinct_count
domain,,
antitrust,126,126
"civil rights, immigration, family",167,167
consumer fraud,200,200
criminal & enforcement,16,16
discrimination,20,20
employment & labor,200,200
environmental & natural resources,3,3
healthcare,9,9
intellectual property & communication,7,7


In [10]:
print(dataset.sample(1).text.iloc[0])


IN THE UNITED STATES DISTRICT COURT 
FOR THE SOUTHERN DISTRICT OF TEXAS 
 CORPUS CHRISTI DIVISION 
  
 KEVIN KING and CHRISTOPHER FRISCO, 
§ 
 Individually and on behalf of all others  
§ 
 similarly situated  
 
 
 
§ 
 
 
  
 
 
 
 
 
 
§ 
 
 
 
 
  
Plaintiffs, 
 
 
 
 
§  
Civil Action No. 2:15-cv-282 
  
 
 
 
 
 
 
§ 
 v. 
 
 
 
 
 
 
§ 
 
  
 
 
 
 
 
 
§ 
JURY TRIAL DEMANDED 
 WASTE FACILITIES, INC. AND MUD   
§ 
 PUDDLES SERVICES, LLC 
 
 
§ 
 
 
 
 
 
 
  
 
 
 
 
 
 
§ 
COLLECTIVE ACTION 
  
Defendants  
 
 
 
§ 
PURSUANT TO 29 U.S.C. § 216(b) 
  
 ORIGINAL COLLECTIVE ACTION COMPLAINT 
  
  
Kevin King and Christopher Frisco bring this action individually and on behalf of 
 all individuals (hereinafter “Plaintiffs and the Potential Class Members” or “Day Rate 
 Workers”) employed by Waste Facilities, Inc. (“WFI”) and/or Mud Puddles Services, LLC 
 (“Mud Puddles”) (collectively, “the Defendants”), at any time during the past three years, 
 to recover compensation, liquidated 

# Start Implement From Here

### Data Preprocessing

### Part #1 - Similarity Calculation

In [ ]:
def calculate_cases_similarity(case_id_a, case_id_b):
    '''
    This method should return a similarity score [0-1] that represents how similar the cases are

    @param case_id_a - the id of the first case
    @param case_id_b - the id of the second case
    @returns a similarity score between the cases
    @rtype float
    '''
    pass

### Part #2 - Multiclass Classification Model

In [ ]:
def train_domain_model(X, y):
    '''
    This method should return a trained classification model

    @param X - pandas dataframe containing all the X values
    @param y - pandas series containing all the y values
    @returns a trained model
    @rtype Pipeline
    '''
    pass

In [ ]:
def test_domain_model(model, X, y):
    '''
    This method should return dictionary with model performance measures

    @param model - Pipeline object model
    @param X - pandas dataframe containing all the X test values
    @param y - pandas series containing all the y true values
    @returns a dictionary with model masures
    @rtype dict
    '''
    pass

In [ ]:
def predict_sample(model, case_id):
    '''
    This method should return a "domain" prediction for this sample

    @param model - Pipeline object model
    @param case_id - the ID of the case we need to predict
    @returns the predicted domain name
    @rtype str
    '''
    pass